## **Convolutional Neural Networks**

Prep data specifically for a CNN
Create a more sophisticated CNN model, understanding a greater variety of model layers
Train a CNN model and observe its performance.
Build and trained a simple model to classify ASL images.

In [1]:
import torch.nn as nn
import pandas as pd
import torch
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

False

Loading the dataset

In [2]:
train_df = pd.read_csv("data/sign_mnist_train.csv")
valid_df = pd.read_csv("data/sign_mnist_valid.csv")

In [3]:
sample_df = train_df.head().copy()  # see the top 5 rows
sample_df.pop('label')
sample_x = sample_df.values
sample_x

array([[107, 118, 127, ..., 204, 203, 202],
       [155, 157, 156, ..., 103, 135, 149],
       [187, 188, 188, ..., 195, 194, 195],
       [211, 211, 212, ..., 222, 229, 163],
       [164, 167, 170, ..., 163, 164, 179]])

In [4]:
sample_x.shape

(5, 784)

In [5]:
IMG_HEIGHT = 28
IMG_WIDTH = 28
IMG_CHS = 1

sample_x = sample_x.reshape(-1, IMG_CHS, IMG_HEIGHT, IMG_WIDTH)
sample_x.shape

(5, 1, 28, 28)

In [6]:
class MyDataset(Dataset):
    def __init__(self, base_df):
        x_df = base_df.copy()
        y_df = x_df.pop('label')
        x_df = x_df.values / 255  # Normalize values from 0 to 1
        x_df = x_df.reshape(-1, IMG_CHS, IMG_WIDTH, IMG_HEIGHT)
        self.xs = torch.tensor(x_df).float().to(device)
        self.ys = torch.tensor(y_df).to(device)

    def __getitem__(self, idx):
        x = self.xs[idx]
        y = self.ys[idx]
        return x, y

    def __len__(self):
        return len(self.xs)

Creating data loader

In [7]:
BATCH_SIZE = 32

train_data = MyDataset(train_df)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
train_N = len(train_loader.dataset)

valid_data = MyDataset(valid_df)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)
valid_N = len(valid_loader.dataset)

In [8]:
batch = next(iter(train_loader))
batch

[tensor([[[[0.4784, 0.5020, 0.5216,  ..., 0.6353, 0.6314, 0.6314],
           [0.4941, 0.5137, 0.5294,  ..., 0.6471, 0.6471, 0.6471],
           [0.5098, 0.5255, 0.5451,  ..., 0.6627, 0.6588, 0.6588],
           ...,
           [0.6706, 0.6824, 0.7020,  ..., 0.8471, 0.8431, 0.8392],
           [0.6706, 0.6902, 0.7059,  ..., 0.8510, 0.8510, 0.8471],
           [0.6706, 0.6863, 0.7059,  ..., 0.8549, 0.8510, 0.8510]]],
 
 
         [[[0.4431, 0.4667, 0.4863,  ..., 0.7529, 0.7490, 0.7490],
           [0.4471, 0.4627, 0.4902,  ..., 0.7529, 0.7529, 0.7529],
           [0.4510, 0.4627, 0.4980,  ..., 0.7569, 0.7608, 0.7608],
           ...,
           [0.5098, 0.5294, 0.5765,  ..., 0.8980, 0.9020, 0.9020],
           [0.5098, 0.5373, 0.5804,  ..., 0.9098, 0.9059, 0.9059],
           [0.5020, 0.5333, 0.5843,  ..., 0.9098, 0.9098, 0.9098]]],
 
 
         [[[0.7098, 0.7216, 0.7294,  ..., 0.7529, 0.7490, 0.7490],
           [0.7216, 0.7412, 0.7529,  ..., 0.7725, 0.7647, 0.7608],
           [0.7255

In [9]:
batch[0].shape

torch.Size([32, 1, 28, 28])

In [10]:
batch[1].shape

torch.Size([32])

#### Creating a Convolutional Model
using batch normalization, maxpool2d, dropout, flattening the output.

In [11]:
n_classes = 24
kernel_size = 3
flattened_img_size = 75 * 3 * 3

model = nn.Sequential(
    # First convolution
    nn.Conv2d(IMG_CHS, 25, kernel_size, stride=1, padding=1),  # 25 x 28 x 28
    nn.BatchNorm2d(25),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 25 x 14 x 14
    # Second convolution
    nn.Conv2d(25, 50, kernel_size, stride=1, padding=1),  # 50 x 14 x 14
    nn.BatchNorm2d(50),
    nn.ReLU(),
    nn.Dropout(.2),
    nn.MaxPool2d(2, stride=2),  # 50 x 7 x 7
    # Third convolution
    nn.Conv2d(50, 75, kernel_size, stride=1, padding=1),  # 75 x 7 x 7
    nn.BatchNorm2d(75),
    nn.ReLU(),
    nn.MaxPool2d(2, stride=2),  # 75 x 3 x 3
    # Flatten to Dense
    nn.Flatten(),
    nn.Linear(flattened_img_size, 512),
    nn.Dropout(.3),
    nn.ReLU(),
    nn.Linear(512, n_classes)
)

In [12]:
model = torch.compile(model.to(device))
model

OptimizedModule(
  (_orig_mod): Sequential(
    (0): Conv2d(1, 25, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(25, 50, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.2, inplace=False)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Conv2d(50, 75, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): BatchNorm2d(75, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU()
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Flatten(start_dim=1, end_dim=-1)
    (14): Linear(in_features=675, out_features=512, bias=True)
    (15): Dropout

In [13]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())

In [14]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

Training model

In [15]:

def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [16]:

def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [18]:
epochs = 10

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()

Epoch: 0
Train - Loss: 1.2221 Accuracy: 0.9996
Valid - Loss: 40.2904 Accuracy: 0.9525
Epoch: 1
Train - Loss: 10.6527 Accuracy: 0.9966
Valid - Loss: 19.9260 Accuracy: 0.9785
Epoch: 2
Train - Loss: 1.4870 Accuracy: 0.9995
Valid - Loss: 211.2135 Accuracy: 0.8279
Epoch: 3
Train - Loss: 4.4204 Accuracy: 0.9985
Valid - Loss: 16.7932 Accuracy: 0.9788
Epoch: 4
Train - Loss: 2.8317 Accuracy: 0.9992
Valid - Loss: 57.5201 Accuracy: 0.9459
Epoch: 5
Train - Loss: 3.3487 Accuracy: 0.9989
Valid - Loss: 14.5140 Accuracy: 0.9749
Epoch: 6
Train - Loss: 5.1286 Accuracy: 0.9984
Valid - Loss: 28.8117 Accuracy: 0.9700
Epoch: 7
Train - Loss: 3.8070 Accuracy: 0.9987
Valid - Loss: 33.9592 Accuracy: 0.9734
Epoch: 8
Train - Loss: 2.0382 Accuracy: 0.9995
Valid - Loss: 14.9603 Accuracy: 0.9802
Epoch: 9
Train - Loss: 3.5345 Accuracy: 0.9990
Valid - Loss: 36.4293 Accuracy: 0.9716
